## Exploratory Analysis - Tests

### Global Imports

In [2]:
### Imports
import os
import json
import pandas as pd

In [3]:
# pip install convokit

### Parameters and Global Variables

In [4]:
# Get current working directory
cwd = os.getcwd()
LOCAL_PATH = cwd.replace("\\", "/")
LOCAL_PATH = LOCAL_PATH.replace("data/statistics", "data/convokit")
LOCAL_PATH

'c:/Users/fdmol/Desktop/MSCAPP/CAPP30254/supreme-court-ml-predictions/supreme_court_predictions/data/convokit'

### Functions

In [5]:
"""
This file serves as the client for convokit.
"""
from convokit import Corpus, download


def get_data():
    """
    Loads and outputs the Supreme Court Corpus data
    """

    print("Loading Supreme Court Corpus Data...")
    corpus = Corpus(filename=download("supreme-corpus"))
    corpus.dump(
        "supreme_corpus", base_path=LOCAL_PATH
    )


In [6]:
### Begin reading data
def load_data(file_name):
    """
    Opens the data and returns it as a dictionary

    :param file_name: The name of the file to open
    :return: The data as a dictionary
    """
    path = LOCAL_PATH + f"/supreme_corpus/{file_name}"
    with open(path) as file:
        data = json.load(file)
    return data

In [36]:
def speakers_to_df(speakers_dict):
    """
    Converts the speakers dictionary to a pandas dataframe

    :param speakers_dict: The speakers dictionary
    :return: The speakers dataframe
    """

    dict_list = []
    for speaker_key in list(speakers_dict.keys()):
        speaker_data = speakers_dict[speaker_key]['meta']
        speaker_data['speaker_key'] = speaker_key
        dict_list.append(speaker_data)

    df = pd.DataFrame(dict_list)
    df.rename(columns={'name': 'speaker_name', 'type': 'speaker_type',
                    'role': 'speaker_role'},
                    inplace=True)
    return df

### Pipeline and Tests

In [37]:
get_data()

Loading Supreme Court Corpus Data...
Dataset already exists at C:\Users\fdmol\.convokit\downloads\supreme-corpus


In [38]:
speakers_dict = load_data("speakers.json")
speakers_df = speakers_to_df(speakers_dict)
speakers_df.head()

,speaker_name,speaker_type,speaker_role,speaker_key
0,Earl Warren,J,justice,j__earl_warren
1,Harry F. Murphy,A,NaN,harry_f_murphy
2,William O. Douglas,J,justice,j__william_o_douglas
3,<INAUDIBLE>,U,inaudible,<INAUDIBLE>
4,Felix Frankfurter,J,justice,j__felix_frankfurter


In [39]:
# Conversations metadata
conversations_dict = load_data("conversations.json")

In [52]:
def get_conversation_dfs(conversations_dict):
    """
    Converts the conversations dictionary to several
    pandas dataframes

    :param conversations_dict: The conversations dictionary
    :return: The conversations dataframe
    """
    metadata_list = []
    advocates_list = []
    voters_list = []

    for conversation_id in list(conversations_dict.keys()):
        clean_dict = {}
        conversation_data = conversations_dict[conversation_id]['meta']
        clean_dict['id'] = conversation_id
        clean_dict['case_id'] = conversation_data['case_id']
        clean_dict['winning_side'] = conversation_data['win_side']

        advocates = conversation_data['advocates']
        voters = conversation_data['votes_side']

        for advocate in advocates:
            advocate_dict = {}
            advocate_dict['id'] = conversation_id
            advocate_dict['case_id'] = conversation_data['case_id']
            advocate_dict['advocate'] = advocate
            advocate_dict['side'] = advocates[advocate]['side']
            advocate_dict['role'] = advocates[advocate]['role']
            advocates_list.append(advocate_dict)

        if voters:
            for voter, vote in voters.items():
                vote_dict = {}
                vote_dict['id'] = conversation_id
                vote_dict['case_id'] = conversation_data['case_id']
                vote_dict['voter'] = voter
                vote_dict['vote'] = vote
                voters_list.append(vote_dict)
        else:
            vote_dict = {}
            vote_dict['id'] = conversation_id
            vote_dict['case_id'] = conversation_data['case_id']
            voters_list.append(vote_dict)
        
        metadata_list.append(clean_dict)


        conversation_metadata_df = pd.DataFrame(metadata_list)
        advocates_df = pd.DataFrame(advocates_list)
        voters_df = pd.DataFrame(voters_list)

        return conversation_metadata_df, advocates_df, voters_df
